# Data selection

Juan Navarro, <jsnavarroa@unal.edu.co>

-------------------------------


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
data_path = os.path.abspath("./data")

## 1. Download the data

Use Selenium to download all the data.

In [10]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.chrome.options.Options()  
#chrome_options.add_argument("--headless")
prefs = {"download.default_directory" : data_path}
chrome_options.add_experimental_option("prefs",prefs)

browser = webdriver.Chrome(chrome_options=chrome_options)
browser.implicitly_wait(30)

def is_element_present(how, what):
    try: 
        browser.find_element(by=how, value=what)
    except NoSuchElementException as e: 
        return False
    return True

def waitForElementPresent(how, what):
    for i in range(60):
        if is_element_present(how, what): 
                break
        time.sleep(1)

# Get the page
browser.get("https://www.superfinanciera.gov.co/ReporteInformacionFIC/faces/reporte/consultaReporte.xhtml")
# Select the search by name
browser.find_element_by_xpath("//table[@id='formaDatosReporte:tipoBusqueda']/tbody/tr/td[3]/div/div[2]/span").click()
# Wait for name field
waitForElementPresent(By.ID, "formaDatosReporte:nombreEntidadInput_input")
# Write name
browser.find_element_by_id("formaDatosReporte:nombreEntidadInput_input").send_keys("FIDUCIARIA BANCOLOMBIA S.A.")
# Accept the first suggestion
browser.find_element_by_xpath("//div[@id='formaDatosReporte:nombreEntidadInput_panel']/ul/li/span").click()
# Select one FIC
browser.find_element_by_xpath("//table[@id='formaDatosReporte:tipoBusquedaFIC']/tbody/tr/td[3]/div/div[2]/span").click()
waitForElementPresent(By.ID, "formaDatosReporte:nombreFICInput_input")
browser.find_element_by_id("formaDatosReporte:nombreFICInput_input").clear()
browser.find_element_by_id("formaDatosReporte:nombreFICInput_input").send_keys("FIDUCUENTA")
browser.find_element_by_xpath("//div[@id='formaDatosReporte:nombreFICInput_panel']/ul/li").click()
# Write initial date
browser.find_element_by_id("formaDatosReporte:fechaReporteInicial_input").send_keys("01/01/2017")
# Write final date
browser.find_element_by_id("formaDatosReporte:fechaReporteFinal_input").send_keys("31/05/2018")
# Begin the search
browser.find_element_by_xpath("//button[@id='formaDatosReporte:cmdReporte']/span").click()

while(True):
    # Download report
    browser.find_element_by_xpath("//button[@id='formaResultadoRentabilidad:j_id_1u']/span[2]").click()
    # Go to next report page
    paginator = browser.find_element_by_xpath("//div[@id='formaResultadoRentabilidad:tblResultadoReporte_paginator_bottom']/span[4]")
    paginator.click()
    if "ui-state-disabled" in paginator.get_attribute("class"):
        break
    time.sleep(1)

## 2. Combine all the data in one DataFrame

In [13]:
import pandas as pd
import glob

report = pd.DataFrame()

# Go to download path and read all files
retval = os.getcwd()
os.chdir(data_path)
for file in glob.glob("reporteRentabilidades*.xls"):
    print(file)
    df = pd.read_excel(file)
    report = report.append(df)

os.chdir(retval)

# Save to file
report.to_csv(data_path+"/profitability.csv", index=False, encoding="utf-8")

report.head()

reporteRentabilidades (11).xls
reporteRentabilidades (9).xls
reporteRentabilidades (16).xls
reporteRentabilidades (14).xls
reporteRentabilidades (12).xls
reporteRentabilidades (2).xls
reporteRentabilidades (7).xls
reporteRentabilidades.xls
reporteRentabilidades (13).xls
reporteRentabilidades (15).xls
reporteRentabilidades (1).xls
reporteRentabilidades (6).xls
reporteRentabilidades (10).xls
reporteRentabilidades (4).xls
reporteRentabilidades (5).xls
reporteRentabilidades (8).xls
reporteRentabilidades (3).xls


,Fecha corte,Tipo Entidad,Cód. Entidad,Nombre Entidad,Cód. Negocio,Nombre Negocio,Subtipo Negocio,Principal / Compart.,Tipo Part. <sup>1<sup/>,Cons. id Part.,Núm. unidades,Valor unidad para las operaciones del día t,Valor fondo al cierre del día t,Núm. Invers.,Rentab. dia,Rentab. mes,Rentab. sem,Rentab. año
0,22/05/2018,5,31,Fiduciaria Bancolombia S.A.,2852,FONDO DE INVERSIÓN COLECTIVA ABIERTO FIDUCUENTA,FIC DE TIPO GENERAL,Principal,8,0,"460,948,399.899","$28,236.00","$12,991,075,451,243.51",508932,4.197424,3.843288,4.257855,4.595863
1,21/05/2018,5,31,Fiduciaria Bancolombia S.A.,2852,FONDO DE INVERSIÓN COLECTIVA ABIERTO FIDUCUENTA,FIC DE TIPO GENERAL,Principal,8,0,"450,917,314.429","$28,232.82","$13,013,871,366,640.45",508930,5.280213,3.820311,4.257999,4.601191
2,20/05/2018,5,31,Fiduciaria Bancolombia S.A.,2852,FONDO DE INVERSIÓN COLECTIVA ABIERTO FIDUCUENTA,FIC DE TIPO GENERAL,Principal,8,0,"450,917,314.434","$28,228.84","$12,728,871,017,061.13",509362,3.298527,3.761271,4.277271,4.600782
3,19/05/2018,5,31,Fiduciaria Bancolombia S.A.,2852,FONDO DE INVERSIÓN COLECTIVA ABIERTO FIDUCUENTA,FIC DE TIPO GENERAL,Principal,8,0,"450,917,314.431","$28,226.33","$12,727,739,316,375.28",507934,3.294377,3.841239,4.276096,4.605794
4,18/05/2018,5,31,Fiduciaria Bancolombia S.A.,2852,FONDO DE INVERSIÓN COLECTIVA ABIERTO FIDUCUENTA,FIC DE TIPO GENERAL,Principal,8,0,"451,568,951.351","$28,223.82","$12,726,609,116,890.71",507934,2.571996,3.919510,4.280969,4.608819
